# Importing Important Libraries 

## Steps to be followed
    1. Importing necessary libraries
    2. Creating S3 bucket
    3. Mapping the path of models in S3

In [2]:
import sagemaker
import boto3 #read s3 buckets if bucket is public
from sagemaker.amazon.amazon_estimator import get_image_uri #deprecated
from sagemaker import image_uris
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name = 'bankapplication16'
my_region = boto3.session.Session().region_name
print(my_region)

eu-north-1


In [4]:
s3 = boto3.resource('s3')
try:
    if my_region == 'eu-north-1':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'eu-north-1'})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [5]:
#set an ouptput path where the trained model will be saved 
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://bankapplication16/xgboost-as-a-built-in-algo/output


# Downloading the dataset and storing in S3

In [6]:
import pandas as pd
import urllib
try:
    #Downloading the dataset from the given url and renaming it to bank_clean.csv
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv')
except Exception as e:
    print('Data load error: ',e)
try:
    #reading the data
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv
Success: Data loaded into dataframe


In [7]:
#Train test split
import numpy as np
'''
This shuffles the dataset (model_data) by using the sample() method. 
The frac=1 means you're taking the full dataset after shuffling. 
The random_state=1729 ensures the shuffling is reproducible, meaning you'll get the same shuffle each time you run this.
'''
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
import os
#Saving Train and Test Data into Buckets
pd.concat([train_data['y_yes'], train_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv('train.csv',
                                                                                          index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}'.format(bucket_name, prefix), content_type='csv')

In [9]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', 
                                                                                          index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

# Building Models Xgboot- Inbuilt Algorithm

In [10]:
#look for the XGBoost image URI and build an XGBoost container
#specify the repo version depending on preference
container = sagemaker.image_uris.retrieve('xgboost',boto3.Session().region_name,version='latest')

In [11]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [17]:
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_type='ml.m5.2xlarge',
                                          instance_count= 1,
                                          volume_size=5,
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [19]:
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2024-10-10-05-25-08-411


2024-10-10 05:25:12 Starting - Starting the training job...
2024-10-10 05:25:27 Starting - Preparing the instances for training...
2024-10-10 05:26:08 Downloading - Downloading the training image......
2024-10-10 05:27:10 Training - Training image download completed. Training in progress.
2024-10-10 05:27:10 Uploading - Uploading generated training modelArguments: train
[2024-10-10:05:27:02:INFO] Running standalone xgboost training.
[2024-10-10:05:27:02:INFO] File size need to be processed in the node: 6.28mb. Available memory size in the node: 23518.96mb
[2024-10-10:05:27:02:INFO] Determined delimiter of CSV input is ','
[05:27:02] S3DistributionType set as FullyReplicated
[05:27:02] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train/train?format=csv&label_column=0&delimiter=,
[2024-10-10:05:27:02:INFO] Determined delimiter of CSV input is ','
[05:27:02] S3DistributionType set as FullyReplicated
[05:27:02] 12357x59 matrix with 729063 entries loaded from /opt/ml/

In [20]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-10-10-05-28-06-706
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-10-10-05-28-06-706
INFO:sagemaker:Creating endpoint with name xgboost-2024-10-10-05-28-06-706


-----!

In [22]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values # Load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [36]:
predictions_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

In [39]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10784)    36% (152)
Purchase        10% (1149)     64% (272) 



# Deleting Endpoints

In [40]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: xgboost-2024-10-10-05-28-06-706


[{'ResponseMetadata': {'RequestId': 'HK18F2206YMA2YRE',
   'HostId': 'HvZyTAUvcI2EVoe8XllfEPAZtgcvDrg6sJnUKFVDLW+FNr+OZ4/ohOPVP1LdeJpwxIQc1+OkFNg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'HvZyTAUvcI2EVoe8XllfEPAZtgcvDrg6sJnUKFVDLW+FNr+OZ4/ohOPVP1LdeJpwxIQc1+OkFNg=',
    'x-amz-request-id': 'HK18F2206YMA2YRE',
    'date': 'Thu, 10 Oct 2024 06:05:22 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-10-10-05-25-08-411/profiler-output/system/incremental/2024101005/1728537960.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-10-10-05-25-08-411/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-10-10-05-25-08-411/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-10-10-05-25-08-411/pro